In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
import numpy as np
import time

# Parameters to tune for memory exhaustion (start small, increase)
batch_size = 100  # Large batch: activations scale with this
input_size = 100  # Wide inputs: increases weight matrices
hidden_units = 1000  # Wide layers: GBs for weights/activations
num_layers = 5  # More layers: deeper activations/gradients
num_classes = 10
num_samples = 5000  # Large dataset: loaded to GPU
epochs = 5  # Run multiple epochs for sustained usage

# Generate synthetic data (large tensors on GPU)
with tf.device('/GPU:0'):
    x_train = tf.random.normal((num_samples, input_size))
    y_train = tf.random.uniform((num_samples,), minval=0, maxval=num_classes, dtype=tf.int32)
    y_train = tf.one_hot(y_train, num_classes)  # One-hot for more memory

# Build large model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(input_size,)))
for _ in range(num_layers):
    model.add(tf.keras.layers.Dense(hidden_units, activation='relu'))  # Dense layers eat memory
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# Compile with optimizer/loss (Adam uses extra memory for states)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Dataset for batched training (prefetch to GPU)
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Training loop to exhaust memory during backprop
print("Starting training...")
start_time = time.time()
history = model.fit(dataset, epochs=epochs, verbose=1)

# Post-training memory check
mem_info = tf.config.experimental.get_memory_info('GPU:0')
print(f"Peak used memory: {mem_info['peak'] / 1024**3:.2f} GB")
print(f"Training time: {time.time() - start_time:.2f} seconds")

# Optional: Release memory
# tf.keras.backend.clear_session()